In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Gastroesophageal_reflux_disease_(GERD)"
cohort = "GSE68698"

# Input paths
in_trait_dir = "../../input/GEO/Gastroesophageal_reflux_disease_(GERD)"
in_cohort_dir = "../../input/GEO/Gastroesophageal_reflux_disease_(GERD)/GSE68698"

# Output paths
out_data_file = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/GSE68698.csv"
out_gene_data_file = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/gene_data/GSE68698.csv"
out_clinical_data_file = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/clinical_data/GSE68698.csv"
json_path = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Molecular characterization of systemic sclerosis esophageal pathology identifies inflammatory and proliferative signatures"
!Series_summary	"Esophageal involvement in patients with systemic sclerosis (SSc) is common, but tissue-specific pathological mechanisms are poorly understood. Fibrosis in the esophagus is thought to disrupt smooth muscle function and lead to esophageal dilatation, but autopsy studies demonstrate esophageal smooth muscle atrophy and the absence of fibrosis in the majority of SSc cases. Molecular characterization of SSc esophageal pathology is lacking. Herein, we perform a detailed characterization of SSc esophageal histopathology and molecular signatures at the level of gene expression. Esophageal biopsies were prospectively obtained during esophagogastroduodenoscopy in 16 consecutive SSc patients and 7 subjects without SSc. Upper and lower esophageal biopsies were evaluated for histopathology and gene expression. Individual 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, the dataset measures gene expression in esophageal biopsies
# from patients with and without systemic sclerosis (SSc).
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Gastroesophageal reflux disease/GERD):
# The sample characteristics indicate case/control data is available at index 0
# The cases are patients with systemic sclerosis which can cause GERD
trait_row = 0

# For age:
# Age information is not present in the sample characteristics dictionary
age_row = None

# For gender:
# Gender information is not present in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert trait value to binary format (0 for control, 1 for case)."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.lower() == "case":
        return 1
    elif value.lower() == "control":
        return 0
    else:
        return None

def convert_age(value):
    """Convert age value to continuous format."""
    # Since age data is not available, this function won't be used,
    # but it's defined for completeness
    if value is None:
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender value to binary format (0 for female, 1 for male)."""
    # Since gender data is not available, this function won't be used,
    # but it's defined for completeness
    if value is None:
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if value in ["female", "f"]:
        return 0
    elif value in ["male", "m"]:
        return 1
    else:
        return None

# 3. Save Metadata - Initial Filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features using the geo_select_clinical_features function
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_df)
    print("Clinical Feature Preview:")
    print(preview)
    
    # Create the directory for the clinical data if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data to a CSV file
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Feature Preview:
{'GSM1679252': [0.0], 'GSM1679253': [0.0], 'GSM1679254': [0.0], 'GSM1679255': [0.0], 'GSM1679256': [0.0], 'GSM1679257': [0.0], 'GSM1679258': [0.0], 'GSM1679259': [0.0], 'GSM1679260': [0.0], 'GSM1679261': [0.0], 'GSM1679262': [0.0], 'GSM1679263': [0.0], 'GSM1679264': [0.0], 'GSM1679265': [1.0], 'GSM1679266': [1.0], 'GSM1679267': [1.0], 'GSM1679268': [1.0], 'GSM1679269': [1.0], 'GSM1679270': [1.0], 'GSM1679271': [1.0], 'GSM1679272': [1.0], 'GSM1679273': [1.0], 'GSM1679274': [1.0], 'GSM1679275': [1.0], 'GSM1679276': [1.0], 'GSM1679277': [1.0], 'GSM1679278': [1.0], 'GSM1679279': [1.0], 'GSM1679280': [1.0], 'GSM1679281': [1.0], 'GSM1679282': [1.0], 'GSM1679283': [1.0], 'GSM1679284': [1.0], 'GSM1679285': [1.0], 'GSM1679286': [1.0], 'GSM1679287': [1.0], 'GSM1679288': [1.0], 'GSM1679289': [1.0], 'GSM1679290': [1.0], 'GSM1679291': [1.0], 'GSM1679292': [1.0], 'GSM1679293': [1.0], 'GSM1679294': [1.0], 'GSM1679295': [1.0], 'GSM1679296': [1.0], 'GSM1679297': [1.0]}
Clinica

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 89
Header line: "ID_REF"	"GSM1679252"	"GSM1679253"	"GSM1679254"	"GSM1679255"	"GSM1679256"	"GSM1679257"	"GSM1679258"	"GSM1679259"	"GSM1679260"	"GSM1679261"	"GSM1679262"	"GSM1679263"	"GSM1679264"	"GSM1679265"	"GSM1679266"	"GSM1679267"	"GSM1679268"	"GSM1679269"	"GSM1679270"	"GSM1679271"	"GSM1679272"	"GSM1679273"	"GSM1679274"	"GSM1679275"	"GSM1679276"	"GSM1679277"	"GSM1679278"	"GSM1679279"	"GSM1679280"	"GSM1679281"	"GSM1679282"	"GSM1679283"	"GSM1679284"	"GSM1679285"	"GSM1679286"	"GSM1679287"	"GSM1679288"	"GSM1679289"	"GSM1679290"	"GSM1679291"	"GSM1679292"	"GSM1679293"	"GSM1679294"	"GSM1679295"	"GSM1679296"	"GSM1679297"
First data line: "A_23_P100001"	-1.834984	-1.82578	-0.599947	-0.132206	0.017572	0.187431	0.430457	0.393148	0.194504	-0.673462	-1.431611	0.289414	0.296018	-0.334698	0.240983	-0.017572	0.096226	0.942176	0.45519	0.281983	0.071123	-0.748888	-1.05681	0.504558	0.087858	-0.834236	-0.154798	-0.123127	-0.366139	0.343739	0.061401	-0.70192	-1.565067	0.058271	-

### Step 4: Gene Identifier Review

In [5]:
# Based on the output, I can see that the gene identifiers start with "A_23_P" followed by numbers
# These are Agilent microarray probe IDs, not standard human gene symbols
# They need to be mapped to standard gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE68698
Line 6: !Series_title = Molecular characterization of systemic sclerosis esophageal pathology identifies inflammatory and proliferative signatures
Line 7: !Series_geo_accession = GSE68698
Line 8: !Series_status = Public on Oct 19 2015
Line 9: !Series_submission_date = May 08 2015
Line 10: !Series_last_update_date = Jan 23 2019
Line 11: !Series_pubmed_id = 26220546
Line 12: !Series_summary = Esophageal involvement in patients with systemic sclerosis (SSc) is common, but tissue-specific pathological mechanisms are poorly understood. Fibrosis in the esophagus is thought to disrupt smooth muscle function and lead to esophageal dilatation, but autopsy studies demonstrate esophageal smooth muscle atrophy


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the ID and gene symbol columns from gene annotation data
# Based on the gene annotation preview, 'ID' contains the probe identifiers (e.g., A_23_P100001)
# and 'GENE_SYMBOL' contains the corresponding gene symbols (e.g., FAM174B)
id_column = 'ID'  # This matches the index in gene_data
gene_symbol_column = 'GENE_SYMBOL'

# 2. Get gene mapping dataframe
# First, extract the full gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file)

# Create the mapping between probe IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, id_column, gene_symbol_column)
print(f"Number of mappings found: {len(mapping_df)}")
print("First few probe-to-gene mappings:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Number of unique genes after mapping: {len(gene_data)}")
print("First few gene symbols:")
print(gene_data.index[:20])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Number of mappings found: 30936
First few probe-to-gene mappings:
             ID     Gene
0  A_23_P100001  FAM174B
1  A_23_P100011    AP3S2
2  A_23_P100022     SV2B
3  A_23_P100056   RBPMS2
4  A_23_P100074     AVEN
Number of unique genes after mapping: 13572
First few gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'AAAS',
       'AACS', 'AADAC', 'AADAT', 'AAGAB', 'AAMP', 'AANAT', 'AARS', 'AARS2',
       'AARSD1', 'AASDH', 'AASDHPPT', 'AASS'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/gene_data/GSE68698.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Load the gene expression data saved in step 6
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
print(f"Loaded gene data shape: {gene_data.shape}")

# Normalize gene symbols using NCBI Gene database
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data (overwrite the previous file with normalized data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data created in step 2
clinical_df = pd.read_csv(out_clinical_data_file)
print(f"Loaded clinical data shape: {clinical_df.shape}")

# If clinical_df doesn't have a proper index, fix it
if 'Unnamed: 0' in clinical_df.columns:
    clinical_df = clinical_df.set_index('Unnamed: 0')
elif not clinical_df.index.name:
    # Just in case the index needs to be set from data
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    clinical_df = clinical_features

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# 3. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
if 'Age' in linked_data.columns:
    print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
if 'Gender' in linked_data.columns:
    print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")

gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
if gene_cols:
    print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")

cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 4. Evaluate bias in trait and demographic features
is_trait_biased = False
if len(cleaned_data) > 0:
    trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
    is_trait_biased = trait_biased
else:
    print("No data remains after handling missing values.")
    is_trait_biased = True

# 5. Final validation and save
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=len(normalized_gene_data) > 0, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=cleaned_data,
    note=f"Dataset contains gene expression data for {trait} analysis."
)

# 6. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Loaded gene data shape: (13572, 46)
Gene data shape after normalization: (13483, 46)
Sample gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'AAAS', 'AACS', 'AADAC', 'AADAT']


Normalized gene data saved to ../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/gene_data/GSE68698.csv
Loaded clinical data shape: (1, 46)
Linked data shape: (46, 13484)
Linked data preview (first 5 rows, first 5 columns):
            Gastroesophageal_reflux_disease_(GERD)      A1BG  A1BG-AS1  \
GSM1679252                                     0.0 -0.467114 -0.046106   
GSM1679253                                     0.0 -0.085174  0.038475   
GSM1679254                                     0.0 -0.002690  0.208568   
GSM1679255                                     0.0  0.034070  0.120268   
GSM1679256                                     0.0  0.394492  0.099820   

                A1CF       A2M  
GSM1679252 -1.033794  0.015503  
GSM1679253 -0.862933  0.224771  
GSM1679254  0.017483 -0.690636  
GSM1679255  1.001625 -0.925642  
GSM1679256  0.389577 -0.378413  

Missing values before handling:
  Trait (Gastroesophageal_reflux_disease_(GERD)) missing: 0 out of 46
  Genes with >20% 

Data shape after handling missing values: (46, 13484)
For the feature 'Gastroesophageal_reflux_disease_(GERD)', the least common label is '0.0' with 13 occurrences. This represents 28.26% of the dataset.
The distribution of the feature 'Gastroesophageal_reflux_disease_(GERD)' in this dataset is fine.



Linked data saved to ../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/GSE68698.csv
